In [15]:
# ===========================================
#  Gujarati Next Word Prediction using Bi-LSTM
#  (Based on "Next Word Prediction in Hindi Using Deep Learning Techniques")
# ===========================================

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
import os

In [32]:
# --- GPU setup ---
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ Enabled memory growth for {len(gpus)} GPU(s).")
    except RuntimeError as e:
        print(f"⚠️ GPU Error: {e}")

✅ Enabled memory growth for 2 GPU(s).


In [27]:
# --- Hyperparameters for Data ---
DATA_PATH = "/kaggle/input/gu-processed/gu_processed.txt"
NUM_LINES = 50000 
VOCAB_TARGET = 30000  # <--- UPDATED: Limit vocab to top 30k words
SEQ_LENGTH = 10       # <--- UPDATED: Increased sequence length for more context

print(f"Loading {NUM_LINES} lines from dataset...")
lines = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= NUM_LINES:
            break
        line = line.strip()
        if line:
            lines.append(line)

print(f"✅ Loaded {len(lines)} lines.")

Loading 50000 lines from dataset...
✅ Loaded 25000 lines.


In [28]:
# --- Text Cleaning ---
def clean_gujarati_text(text):
    text = re.sub(r'[^\u0A80-\u0AFF\s]', '', text)  # keep Gujarati chars
    text = re.sub(r'\s+', ' ', text).strip()
    return text

cleaned_text = ' '.join(clean_gujarati_text(line) for line in lines)
print(f"Sample cleaned text: {cleaned_text[:150]}...")

# --- Tokenization ---
# <--- UPDATED: Apply vocab limit and OOV token
tokenizer = Tokenizer(num_words=VOCAB_TARGET, oov_token="<UNK>") 
tokenizer.fit_on_texts([cleaned_text])

# Save tokenizer
with open("gujarati_tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# <--- UPDATED: The vocab_size for the model is now our target
vocab_size = VOCAB_TARGET 
print(f"✅ Vocabulary Size set to: {vocab_size}")

# --- Create sequences (10:1 ratio) ---
tokens = tokenizer.texts_to_sequences([cleaned_text])[0]
sequences = []

# <--- UPDATED: Loop will use new SEQ_LENGTH
for i in range(SEQ_LENGTH, len(tokens)): 
    seq = tokens[i-SEQ_LENGTH:i+1]
    sequences.append(seq)

sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]

print(f"Total sequences: {len(X)}")
print(f"Input shape: {X.shape}, Output shape: {y.shape}")

Sample cleaned text: આ વીડિયો જુઓ ઊંઝા માર્કેટયાર્ડ આજથી જુલાઈ સુધી બંધ મિથેનોલ આવ્યો ક્યાંથી આખરે ત્રણ રાજ્યોમાં મળેલ હાર પર કોંગ્રેસ અધ્યક્ષ રાહુલ ગાંધી દ્વારા પ્રથમ પ્ર...
✅ Vocabulary Size set to: 30000
Total sequences: 1020578
Input shape: (1020578, 10), Output shape: (1020578,)


In [29]:
# --- Create Train/Test Split ---
# <--- NEW: Added train/test split, which model.fit() needs
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train shapes: {X_train.shape}, {y_train.shape}")
print(f"Test shapes: {X_test.shape}, {y_test.shape}")

Train shapes: (816462, 10), (816462,)
Test shapes: (204116, 10), (204116,)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# --- Tuned Model Hyperparameters ---
# These must match the values from Part 1
# vocab_size = 30000 
# SEQ_LENGTH = 10

EMBED_DIM = 250       # <--- UPDATED: Increased embedding dim
LSTM_UNITS = 256      # <--- UPDATED: Increased LSTM capacity
DENSE_UNITS = 512     # <--- UPDATED: Increased Dense layer capacity
LR = 0.001            # <--- UPDATED: Slightly higher start LR
EPOCHS = 100
BATCH_SIZE = 128      # 128 or 256 is good

model = Sequential([
    Embedding(input_dim=vocab_size, 
              output_dim=EMBED_DIM, 
              input_length=SEQ_LENGTH),
    
    # <--- UPDATED: Added internal dropout
    Bidirectional(LSTM(LSTM_UNITS, 
                       return_sequences=True,
                       dropout=0.2,            
                       recurrent_dropout=0.2)),
    
    # <--- UPDATED: Removed separate Dropout, added internal dropout
    Bidirectional(LSTM(LSTM_UNITS,
                       dropout=0.2, 
                       recurrent_dropout=0.2)),
    
    Dense(DENSE_UNITS, activation='relu'),
    Dropout(0.5),  # <--- UPDATED: Stronger dropout before final layer
    
    Dense(vocab_size, activation='softmax')
])

optimizer = Adam(learning_rate=LR)
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy'])
model.build(input_shape=(None, SEQ_LENGTH))
model.summary()

# --- Callbacks ---
early_stopper = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
# <--- UPDATED: Lowered min_lr
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=1e-6) 

print("\n🚀 Starting Tuned Bi-LSTM Training...\n")
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopper, reduce_lr],
    verbose=1
)

# --- Evaluate ---
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"\n🎯 Final Test Accuracy: {test_acc:.4f}")
print(f"🧾 Final Test Loss: {test_loss:.4f}")

# --- Save Model ---
model.save("bilstm_gujarati_tuned_model.keras")
print("💾 Saved: bilstm_gujarati_tuned_model.keras")

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_13 (Embedding)        │ (None, 10, 250)        │     7,500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_16                │ (None, 10, 512)        │     1,038,336 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_17                │ (None, 512)            │     1,574,912 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 30000)          │    15,390,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,765,904 (98.29 MB)

 Trainable params: 25,765,904 (98.29 MB)

 Non-trainable params: 0 (0.00 B)


🚀 Starting Tuned Bi-LSTM Training...

Epoch 1/100
6379/6379 ━━━━━━━━━━━━━━━━━━━━ 786s 121ms/step - accuracy: 0.1203 - loss: 7.6329 - val_accuracy: 0.1428 - val_loss: 7.1239 - learning_rate: 0.0010
Epoch 2/100
6379/6379 ━━━━━━━━━━━━━━━━━━━━ 769s 121ms/step - accuracy: 0.1450 - loss: 7.0320 - val_accuracy: 0.1526 - val_loss: 6.9115 - learning_rate: 0.0010
Epoch 3/100
6379/6379 ━━━━━━━━━━━━━━━━━━━━ 771s 121ms/step - accuracy: 0.1558 - loss: 6.7354 - val_accuracy: 0.1600 - val_loss: 6.7971 - learning_rate: 0.0010
Epoch 4/100
6379/6379 ━━━━━━━━━━━━━━━━━━━━ 774s 121ms/step - accuracy: 0.1636 - loss: 6.5273 - val_accuracy: 0.1643 - val_loss: 6.7215 - learning_rate: 0.0010
Epoch 5/100
6379/6379 ━━━━━━━━━━━━━━━━━━━━ 779s 122ms/step - accuracy: 0.1733 - loss: 6.3396 - val_accuracy: 0.1677 - val_loss: 6.6823 - learning_rate: 0.0010
Epoch 6/100
6379/6379 ━━━━━━━━━━━━━━━━━━━━ 775s 122ms/step - accuracy: 0.1796 - loss: 6.1889 - val_accuracy: 0.1712 - val_loss: 6.6703 - learning_rate: 0.0010
Epoch 7

In [ ]:
# --- Plot Accuracy and Loss ---
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Gujarati Bi-LSTM Accuracy (Paper Architecture)")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Gujarati Bi-LSTM Loss (Paper Architecture)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()